In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
os.chdir('IMERG')

In [7]:
train = pd.read_csv("ALL_TRAIN_DATA_RESAMPLE.csv")
train = train[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(train.GIS_ID))
train.head()

58112


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_200001_TD_2000060806,25,25,25
1,ATL_200001_TD_2000060812,25,25,25
2,ATL_200002_TD_2000062312,30,30,25
3,ATL_200002_TD_2000062318,30,30,30
4,ATL_200002_TD_2000062400,30,30,30


In [8]:
test = pd.read_csv("ALL_TEST_DATA.csv")
test = test[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(test.GIS_ID))
test.head()

4312


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_201801_TS_2018052812,50,55,55
1,ATL_201801_TS_2018052818,45,50,55
2,ATL_201801_TD_2018052900,30,45,50
3,ATL_201801_TD_2018052906,30,30,45
4,ATL_201801_TD_2018052912,30,30,30


In [9]:
a = pd.read_csv("IMERG_CSV/" + train.GIS_ID[0] + '.csv', header = None)
print(a.head())
pvmax = np.array([train.VMAX_N06[0], train.VMAX_N12[0]])
print(pvmax)

        0         1         2         3         4         5    6         7    \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
2  0.000000  0.000000  0.002614  0.035389  0.031253  0.003356  0.0  0.000000   
3  0.053374  0.086443  0.075739  0.183154  0.187500  0.022317  0.0  0.046795   
4  0.312158  0.434224  0.340077  0.413843  0.241398  0.028628  0.0  0.202615   

        8         9    ...       111       112       113  114  115  116  117  \
0  0.000000  0.000000  ...  0.327364  0.000000  0.000000  0.0  0.0  0.0  0.0   
1  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
2  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
3  0.354221  0.840079  ...  0.000000  0.295887  0.192012  0.0  0.0  0.0  0.0   
4  1.544581  4.629189  ...  0.100727  0.147943  0.047216  0.0  0.0  0.0  0.0   

   118  119  120  
0  0.0  0.0  0.0  


In [10]:
train_img = []
train_vmax = []
train_label = []
test_img = []
test_vmax = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = f"IMERG_CSV/{train.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91]
        temp = temp.iloc[:, 30:91]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.VMAX[f]
        train_label.append(lab)
        pvmax = np.array([train.VMAX_N06[f], train.VMAX_N12[f]])
        train_vmax.append(pvmax)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = f"IMERG_CSV/{test.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91]
        temp = temp.iloc[:, 30:91]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.VMAX[f]
        test_label.append(lab)
        pvmax = np.array([test.VMAX_N06[f], test.VMAX_N12[f]])
        test_vmax.append(pvmax)
    except Exception as e:
        pass

In [11]:
print(len(train_img))
print(len(train_vmax))
print(len(train_label))
print(len(test_img))
print(len(test_vmax))
print(len(test_label))

58112
58112
58112
4312
4312
4312


In [13]:
X_train_img = train_img
X_train_vmax = train_vmax
y_train = train_label
X_test_img = test_img
X_test_vmax = test_vmax
y_test = test_label

In [14]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,61,61,1)
X_train_img = X_train_img.astype('float32')
X_train_vmax = np.array(X_train_vmax)
X_train_vmax = X_train_vmax.reshape(-1,2)
y_train = np.array(y_train)

In [15]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,61,61,1)
X_test_img = X_test_img.astype('float32')
X_test_vmax = np.array(X_test_vmax)
X_test_vmax = X_test_vmax.reshape(-1,2)
y_test = np.array(y_test)

In [16]:
# 300km
vmax_input = keras.Input(shape =(2,), name = "vmax_layer")
img_input = keras.Input(shape =(61, 61, 1), name = "img_layer")

w = keras.layers.Conv2D(64,12) (img_input)
w = keras.layers.Conv2D(64,12)(w)
w = keras.layers.Conv2D(64,2)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.linear(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(256,2)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, vmax_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(170, activation = 'linear')(output_layer1)

new_model1 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 12) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,2, activation = 'linear')(x)
x = keras.layers.Conv2D(128,7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,2, activation = 'linear')(x)
x = keras.layers.Conv2D(64,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, vmax_input])
output_layer2 = keras.layers.Dense(170, activation = 'linear')(merged_model2)

new_model2 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (10,4))(img_input)
y = keras.layers.Conv2D(256, (4,10))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,6)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, vmax_input])
output_layer3 = keras.layers.Dense(170, activation = 'linear')(merged_model3)

new_model3 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

# z = keras.layers.Conv2D(256, 10)(img_input)
# z = keras.layers.MaxPool2D(2,2)(z)
# img_output4 = keras.layers.Flatten()(z)

# merged_model4 = keras.layers.concatenate([img_output4, vmax_input])
# output_layer4 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model4)

# new_model4 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer4, name = "model_4")

# new_model4.summary()

2023-09-05 15:40:58.289098: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-09-05 15:40:58.289118: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-09-05 15:40:58.289123: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-09-05 15:40:58.289154: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-05 15:40:58.289167: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_layer (InputLayer)      [(None, 61, 61, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 50, 50, 64)           9280      ['img_layer[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)           (None, 39, 39, 64)           589888    ['conv2d[0][0]']              
                                                                                                  
 conv2d_2 (Conv2D)           (None, 38, 38, 64)           16448     ['conv2d_1[0][0]']            
                                                                                            

In [17]:
models = {new_model1: '1', new_model2: '2', new_model3: '3'}
ep = range(10, 11)
batches = [1, 2, 4, 8, 16]
for m in models.keys():
    for e in ep:
        for b in batches:
            with tf.device('/GPU:0'):
                m.compile(optimizer = 'adam', loss = tf.keras.losses.MeanAbsoluteError(), metrics = ['mae', 'mse'])
                m.fit([X_train_img, X_train_vmax], y_train, epochs = e, batch_size = b)
                tf.keras.models.save_model(m, f"MODELS/MODEL{models[m]}_EPOCHS{e}_BATCH{b}")
                res = m.evaluate([X_test_img, X_test_vmax], y_test)
                print("MAE = " + str(res[0]))
                print("RMSE = " + str((res[2]) ** 0.5))
                preds = m.predict([X_test_img, X_test_vmax])
                x = np.average(preds, axis = 1)
                test['preds'] = x
                test.to_csv(f"OUTPUT/MODEL{models[m]}_RESULTS{e}_{b}.csv")
                tf.keras.backend.clear_session()

Epoch 1/10


2023-09-05 15:40:59.832895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


58112/58112 [==============================] - 525s 9ms/step - loss: 4.9139 - mae: 4.9139 - mse: 51.6830
Epoch 2/10
58112/58112 [==============================] - 523s 9ms/step - loss: 4.5203 - mae: 4.5203 - mse: 43.1074
Epoch 3/10
58112/58112 [==============================] - 524s 9ms/step - loss: 4.4474 - mae: 4.4474 - mse: 42.0894
Epoch 4/10
58112/58112 [==============================] - 523s 9ms/step - loss: 4.4193 - mae: 4.4193 - mse: 41.6192
Epoch 5/10
58112/58112 [==============================] - 523s 9ms/step - loss: 4.3862 - mae: 4.3862 - mse: 41.3240
Epoch 6/10
58112/58112 [==============================] - 524s 9ms/step - loss: 4.3760 - mae: 4.3760 - mse: 40.9692
Epoch 7/10
58112/58112 [==============================] - 524s 9ms/step - loss: 4.3575 - mae: 4.3575 - mse: 40.7722
Epoch 8/10
58112/58112 [==============================] - 523s 9ms/step - loss: 4.3465 - mae: 4.3465 - mse: 40.5956
Epoch 9/10
58112/58112 [==============================] - 523s 9ms/step - loss: 4.3

INFO:tensorflow:Assets written to: MODELS/MODEL1_EPOCHS10_BATCH1/assets
2023-09-05 17:08:15.554559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


135/135 [==============================] - 2s 11ms/step - loss: 3.7798 - mae: 3.7798 - mse: 27.4982
MAE = 3.7798173427581787
RMSE = 5.243873836614593
  7/135 [>.............................] - ETA: 1s 

...

3632/3632 [==============================] - 411s 113ms/step - loss: 1.8361 - mae: 1.8361 - mse: 10.9140
Epoch 2/10
3632/3632 [==============================] - 412s 114ms/step - loss: 1.7938 - mae: 1.7938 - mse: 10.6383
Epoch 3/10
3632/3632 [==============================] - 410s 113ms/step - loss: 1.7735 - mae: 1.7735 - mse: 10.3981
Epoch 4/10
3632/3632 [==============================] - 411s 113ms/step - loss: 1.7599 - mae: 1.7599 - mse: 10.3400
Epoch 5/10
3632/3632 [==============================] - 412s 114ms/step - loss: 1.7430 - mae: 1.7430 - mse: 10.1182
Epoch 6/10
3632/3632 [==============================] - 412s 113ms/step - loss: 1.7267 - mae: 1.7267 - mse: 10.0597
Epoch 7/10
3632/3632 [==============================] - 413s 114ms/step - loss: 1.7253 - mae: 1.7253 - mse: 9.9937
Epoch 8/10
3632/3632 [==============================] - 411s 113ms/step - loss: 1.7012 - mae: 1.7012 - mse: 9.8502
Epoch 9/10
3632/3632 [==============================] - 411s 113ms/step - loss: 1.703

INFO:tensorflow:Assets written to: MODELS/MODEL3_EPOCHS10_BATCH16/assets
2023-09-06 04:34:23.302897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


135/135 [==============================] - 10s 72ms/step - loss: 4.4508 - mae: 4.4508 - mse: 37.2774
MAE = 4.450841903686523
RMSE = 6.105524669349647
  3/135 [..............................] - ETA: 8s

2023-09-06 04:34:33.163077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


135/135 [==============================] - 9s 70ms/step


In [43]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [44]:
new_model1.fit([X_train_img, X_train_vmax], y_train, epochs=1, batch_size=1, validation_split = 0.1)
res = new_model1.evaluate([X_test_img, X_test_vmax], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = new_model1.predict(X_test_img)

  354/29703 [..............................] - ETA: 41:24 - loss: 55.2345 - mae: 55.2345 - mse: 3897.4333

KeyboardInterrupt: 

In [ ]:
x = np.average(preds, axis = 1)
x.size

In [ ]:
test['preds'] = x
test

In [ ]:
test.to_csv("OUTPUT/MODEL1_RESULTS.csv")

In [ ]:
model1 = keras.Sequential([
   keras.layers.Conv2D(64,5, activation = 'relu', input_shape = (24,24,1)),
   keras.layers.Conv2D(64,5, activation = 'relu'),
   keras.layers.Conv2D(64,1, activation = 'relu'),
   keras.layers.MaxPool2D((2,2), strides = 2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(64, 4, activation='relu'),
   keras.layers.Conv2D(64, 4, activation='relu'),
   keras.layers.Conv2D(256,1, activation = 'relu'),
   keras.layers.BatchNormalization(),
   keras.layers.Flatten(),
   keras.layers.Dense(256),
   keras.layers.Dense(165)
])

model2 = keras.Sequential([
   keras.layers.Conv2D(256, 5, activation='relu', input_shape=(24,24,1)),
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(128, 1, activation = 'relu'),
   keras.layers.Conv2D(128,3, activation = 'relu'), 
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Conv2D(64, 1, activation = 'relu'),
   keras.layers.Conv2D(64,2, activation = 'relu'), 
   keras.layers.MaxPool2D(2,2),
   keras.layers.BatchNormalization(),
   keras.layers.Flatten(),
   keras.layers.Dense(165, activation = 'linear')
])
model3 = keras.Sequential([
    keras.layers.Conv2D(256, (5,2), activation='relu', input_shape=(24,24,1)),
    keras.layers.Conv2D(256, (2,5), activation = 'relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128,3, activation = 'relu'), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64,2, activation = 'relu'), 
    keras.layers.MaxPool2D(2,2),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(165)
])
model4 = keras.Sequential([
    keras.layers.Conv2D(256, (5,5), activation = 'relu', input_shape = (24,24,1)), 
    keras.layers.MaxPool2D(2,2), 
    keras.layers.Flatten(), 
    keras.layers.Dense(165)
])


model1.summary()
model2.summary()
model3.summary()
model4.summary()

In [ ]:
model3.fit(X_train_img, y_train, epochs=3, batch_size=32, validation_split = 0.1)
res = model3.evaluate(X_test_img, y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))